In [1]:
from wspcfactory import * 
from wsfalabella import *
import pandas as pd 
import numpy as np 
import re
from statistics import mode

In [2]:
#Extraccion de datos desde pagina web de pc factory
df_factory = web_scraping_pc_factory(n_pages=2)
df_factory.to_csv('data/df_factory.csv',index=False)

Web Scraping finished:
Data extracted: 90


In [3]:
#Extraccion de datos desde pagina web de Falabella
df_falabella = web_scraping_falabella(n_pages=8)
df_falabella.to_csv('data/df_falabella.csv',index=False)

Running Web Scraping....

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
 
[WDM] - Driver [C:\Users\crist\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache
Web Scraping finished:
Data extracted: 226


In [4]:
#Funcion para limpiar formato de datos de pc factory 
def clean_pcfactory(df):
    df.titulo = df.titulo.str.replace('Notebook',"").str.strip()
    df.procesador = df.procesador.str.replace('Procesador:',"").str.strip()
    df.tarjeta_grafica = df.tarjeta_grafica.str.replace('Tarjeta Gráfica:',"").str.strip()
    df.marca = df.marca.str.replace('Marca:',"").str.strip()
    df.sistema_operativo = df.sistema_operativo.str.replace('Sistema Operativo:',"").str.strip()

    df.ram = df.ram.str.split("GB").str[0].str.strip()
    df.ram = df.ram.str.replace(r'Memoria\s*\w*:',"").str.strip()
    
    df.almacenamiento = df.almacenamiento.str.extract(r'(\d+GB|\d+TB|\d+\s*TB)',expand=False)
    df.almacenamiento = df.almacenamiento.str.replace(r"GB|\xa0","").str.replace(r"1TB|1 TB","1000").str.replace(r"2TB|2 TB","2000")
    df.almacenamiento = df.almacenamiento.str.replace('Almacenamiento:',"").str.strip()
    
    df.pantalla = df.pantalla.str.replace(r'Pantalla\s*\w*:',"").str.replace(",",".").str.strip()
    df.pantalla = df.pantalla.str.extract(r"(^\d*\.*\d*)")

    df.precio_normal = df.precio_normal.str.replace(r'\$|\.',"").str.strip()
    df.precio_oferta = df.precio_oferta.str.replace(r'\$|\.',"").str.strip()
    return df

df_factory = clean_pcfactory(df_factory)

In [5]:
#Cambiar tipo de datos de pc factory
df_factory.ram = df_factory.ram.astype('float')
df_factory.precio_normal = df_factory.precio_normal.astype('int')
df_factory.precio_oferta = df_factory.precio_oferta.astype('int')
df_factory.almacenamiento = df_factory.almacenamiento.astype('float')
df_factory.pantalla = df_factory.pantalla.astype('float')

In [6]:
#Funcion para limpiar formato de datos de falabella
def clean_falabella(df):

    df.precio_normal = df.precio_normal.str.replace(r"\$|\.","").str.replace(r"\(Oferta\)","").str.strip()
    df.precio_oferta = df.precio_oferta.str.replace(r"\$|\.","").str.replace(r"\(Oferta\)","").str.strip()

    cols_to_edit = ['procesador','ram','tarjeta_grafica','pantalla','almacenamiento','sistema_operativo']

    df.almacenamiento = df.almacenamiento.str.replace("1tb","1000").str.replace("2tb","2000").str.strip()


    for column in cols_to_edit:
        df[column] = df[column].str.lower()
        df[column] = df[column].str.replace(r"|gb|mb|pulgada\w*","").str.strip()
        df[column] = df[column].str.replace(r"[Ss]in pantalla","no aplica").str.strip()

    df = df.replace(r'no aplica',np.nan)
   
    return df 

In [7]:
#ejecutar dos veces para limpiar todos los campos de falabella
df_falabella = clean_falabella(df_falabella)
df_falabella = clean_falabella(df_falabella)

In [8]:
#Cambiar tipo de datos de falabella
df_falabella.ram = df_falabella.ram.astype('float')
df_falabella.pantalla = df_falabella.pantalla.astype('float')
df_falabella.almacenamiento = df_falabella.almacenamiento.astype('float')
df_falabella.precio_normal = df_falabella.precio_normal.astype('float')
df_falabella.precio_oferta = df_falabella.precio_oferta.astype('float')

In [9]:
#Funcion para remplazar valores nulos con la moda de cada columna
def replace_nan_with_mode_factory(df):
    for column in df.columns:
        temp_mode = mode(df[column].tolist())
        df[column] = df[column].fillna(temp_mode)

    return df

df_factory = replace_nan_with_mode_factory(df_factory)

In [10]:
#Funcion para remplazar valores nulos con la moda de cada columna
def replace_nan_with_mode_falabella(df):
    df.precio_oferta = df.precio_oferta.fillna(df.precio_oferta.mean())
    temp_df = df[df.precio_normal.notnull()]
    variation_price = mode((temp_df.precio_normal - temp_df.precio_oferta))
  
    temp_nan_df = df[df.precio_normal.isnull()]
    for i in temp_nan_df.index:
        df.loc[i,'precio_normal'] = df.loc[i,'precio_oferta'] + variation_price

    for i in df.index:
        if df.loc[i,'precio_oferta'] > df.loc[i,'precio_normal']:
            df.loc[i,'precio_normal'] = df.loc[i,'precio_oferta'] + variation_price

    
    temp_df = df[df.tarjeta_grafica.notnull()]
    graphics_mode = mode(temp_df.tarjeta_grafica)
    temp_df1 = df[df['sistema_operativo'].notnull()]
    system_mode = mode(temp_df1['sistema_operativo'])

    for column in df.columns[:-2]:
        if column == 'tarjeta_grafica':
            df[column] = df[column].fillna(graphics_mode)
  
        if column == 'sistema_operativo':
            df[column] = df[column].fillna(system_mode)

        else:
            df[column] = df[column].fillna(mode(df[column].tolist()))

    return df
    
df_falabella = replace_nan_with_mode_falabella(df_falabella)

In [11]:
df_factory.isnull().sum()

compania             0
titulo               0
marca                0
procesador           0
ram                  0
tarjeta_grafica      0
pantalla             0
almacenamiento       0
sistema_operativo    0
precio_normal        0
precio_oferta        0
urls                 0
dtype: int64

In [12]:
df_falabella.isnull().sum()

compania             0
titulo               0
marca                0
procesador           0
ram                  0
tarjeta_grafica      0
pantalla             0
almacenamiento       0
sistema_operativo    0
precio_normal        0
precio_oferta        0
urls                 0
dtype: int64

In [13]:
df_factory.to_csv('data/clean_pcfactory.csv',index=False)
df_falabella.to_csv('data/clean_falabella.csv',index=False)